In [1]:
## Imports

import pandas as pd
import io
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_auc_score

import re
import string
import nltk
import itertools
import numpy as np

# this allows you to apply a progress bar to pandas operations
from tqdm import tqdm
tqdm.pandas()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Github repo installation of paddle
!python -m pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple
# Install paddle OCR
!pip install paddleocr
# git clone
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
# cv2
!pip install "opencv-python-headless<4.3"


Looking in indexes: https://mirror.baidu.com/pypi/simple
fatal: destination path 'PaddleOCR' already exists and is not an empty directory.


In [3]:
import cv2
from paddleocr import PaddleOCR, draw_ocr
from matplotlib import pyplot as plt
import os

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': Image.NEAREST,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': Image.BILINEAR,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': Image.BICUBIC,
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/paddle/vision/transforms/functional_pil.py:39: DeprecationWarning: BOX is deprecated and will be removed in Pillo

In [4]:
ocr_model = PaddleOCR(lang='en')

def extract_text(image_path):
    result = ocr_model.ocr(image_path)
    
    return [res[1][0] for res in result]

[2022/09/02 13:54:17] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, shape_info_filename=None, precision='fp32', gpu_mem=500, image_dir=None, det_algorithm='DB', det_model_dir='/home/studio-lab-user/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='SVTR_LCNet', rec_model_dir='/home/studio-lab-user/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer', rec_image_shape='3, 48, 320', rec_bat

---    Fused 0 subgraphs into layer_norm op.
---    Fused 0 subgraphs into layer_norm op.


In [110]:
# To pull in previusly save data
import pandas as pd
data = pd.read_csv("easy_results.csv", encoding='latin')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'easy_results.csv'

In [111]:
%cd /home/studio-lab-user/DS-Projects/OCR/OCR_image_test

/home/studio-lab-user/DS-Projects/OCR/OCR_image_test


In [155]:
image_dff = data[['image','relevant']]

In [156]:
image_dff

,image,relevant
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1
1,68ece3fab138feb537eee296f3348f3543e23a1d,0
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0
3,a278812a832f24a822e2602b91a7b1f5749926cb,1
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0
5,9a12a490fcea7313378b1e83425fae86ddc81124,0
6,533a514c8c422f93173db706aeb8a2e87603e575,1
7,7dc026543aaf6863f0bc810acda21e888cec7693,0
8,c24c8919bfeb37dbca1422e373d6f79693efedcd,0
9,c74d92fc4f342df4e9a7bf9db1b77ad5cfeea89c,0


In [157]:
# # OCR_images에 있는 리스트들로 컬럼 하나 만들기. 리스트 이름 따기.
# import pandas as pd
# import os
# image_df = pd.DataFrame(os.listdir(), columns=['image'])

In [158]:
# image_dff
# image_dff.drop(index=0, inplace=True)

In [116]:
image_list = image_dff['image'].tolist()
len(image_list)

469

In [117]:
# text_list = []
# for image_path in image_list:
#     result = ocr_model.ocr(image_path)
#     text = [res[1][0] for res in result]
#     text_list.append(text)
# print(text_list)

In [313]:
image_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/000e7696e9936c1f2d2e2214cdfbdb54b8878374.jpg"


In [407]:
image_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/000e7696e9936c1f2d2e2214cdfbdb54b8878374.jpg"
# result = ocr_model.ocr(image_path)
texts = [res[1][0] for res in result]
texts

['VWoud n2', 'AMONG US', 'IMPOSTOR']

In [408]:
final = []
for word in texts:
    final.extend(word.split())
final

['VWoud', 'n2', 'AMONG', 'US', 'IMPOSTOR']

In [413]:
def get_words_and_scores_and_add_to_dataframe(row):
    photo = row["image"]
    if photo == "empty":
        return [[],[]]
    final = []
    image_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/{photo}.jpg"
    result = ocr_model.ocr(image_path)
    texts = [res[1][0] for res in result]
    for word in texts:
        final.extend(word.split())
    scores = [res[1][1] *100 for res in result]
    result = [final, scores]
    return result

In [416]:
# image_dff_100 = image_dff[100:150]

In [430]:
image_dff["words"] = image_dff.apply(get_words_and_scores_and_add_to_dataframe, axis=1)
image_dff

[2022/09/02 16:51:01] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/02 16:51:02] ppocr DEBUG: dt_boxes num : 2, elapse : 0.7814555168151855
[2022/09/02 16:51:02] ppocr DEBUG: rec_res num  : 2, elapse : 0.2824568748474121
[2022/09/02 16:51:02] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/02 16:51:03] ppocr DEBUG: dt_boxes num : 1, elapse : 1.1150290966033936
[2022/09/02 16:51:03] ppocr DEBUG: rec_res num  : 1, elapse : 0.12218713760375977
[2022/09/02 16:51:03] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2022/09/02 16:51:03] ppocr DEBUG: dt_boxes num : 0, elapse : 0.08868122100830078
[2022/09/02 16:51:03] ppocr DEBUG: rec_res num  : 0, elapse : 1.6689300537109375e-06
[2022/09/02 16:51:03] ppocr WARNING: Since the angle classifi

,image,relevant,words
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1,"[[AMONGUS], [82.79706239700317]]"
1,68ece3fab138feb537eee296f3348f3543e23a1d,0,"[[zfsz.en.alibaba.com], [92.13441610336304]]"
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0,"[[], []]"
3,a278812a832f24a822e2602b91a7b1f5749926cb,1,"[[Impostor], [97.47896194458008]]"
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0,"[[Press, to, make, a, sound], [99.687016010284..."
5,9a12a490fcea7313378b1e83425fae86ddc81124,0,"[[], []]"
6,533a514c8c422f93173db706aeb8a2e87603e575,1,"[[AMONG-US, AMONG, AMONC, US, YOU, YOU, LOOK, ..."
7,7dc026543aaf6863f0bc810acda21e888cec7693,0,"[[changing, Face], [96.80294394493103, 98.2673..."
8,c24c8919bfeb37dbca1422e373d6f79693efedcd,0,"[[], []]"
9,c74d92fc4f342df4e9a7bf9db1b77ad5cfeea89c,0,"[[], []]"


In [431]:
import re
import string

def clean_text(row):
    word_list = row["words"][0]
    new_word_list = []
    for word in word_list:
        new_word = word.lower()
        new_word = re.sub("[%s]" % re.escape(string.punctuation), " ", new_word)
        new_word_list.append(new_word)
    return [new_word_list, row["words"][1]]

In [432]:
image_dff["words"] = image_dff.apply(clean_text, axis=1)

In [433]:
image_dff

,image,relevant,words
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1,"[[amongus], [82.79706239700317]]"
1,68ece3fab138feb537eee296f3348f3543e23a1d,0,"[[zfsz en alibaba com], [92.13441610336304]]"
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0,"[[], []]"
3,a278812a832f24a822e2602b91a7b1f5749926cb,1,"[[impostor], [97.47896194458008]]"
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0,"[[press, to, make, a, sound], [99.687016010284..."
5,9a12a490fcea7313378b1e83425fae86ddc81124,0,"[[], []]"
6,533a514c8c422f93173db706aeb8a2e87603e575,1,"[[among us, among, amonc, us, you, you, look, ..."
7,7dc026543aaf6863f0bc810acda21e888cec7693,0,"[[changing, face], [96.80294394493103, 98.2673..."
8,c24c8919bfeb37dbca1422e373d6f79693efedcd,0,"[[], []]"
9,c74d92fc4f342df4e9a7bf9db1b77ad5cfeea89c,0,"[[], []]"


In [526]:
## define the target words / phrases

targets = ["among us", "impostor", "among", "amongus"]

In [527]:
# helper functions for funal scoring function

def advance(iterator, step):
    next(itertools.islice(iterator, step, step), None)


def tuplewize(iterable, size):
    iterators = itertools.tee(iterable, size)
    for position, iterator in enumerate(iterators):
        advance(iterator, position)
    return zip(*iterators)

In [528]:
# define function to produce final scores

def get_final_score(row):
    sample_list = row["words"][0]
    score_list = row["words"][1]
    max_score = 0
    
    for target in targets:
        phrase_length = len(target.split())
        for c, s in zip(tuplewize(sample_list, phrase_length), tuplewize(score_list, phrase_length)):
            new_sample = " ".join(c)
            edit_distance = nltk.edit_distance(target, new_sample)
            closeness = (len(target)-edit_distance)/len(target)
            confidence_score = np.mean(s)
            final_score = closeness * confidence_score
            if final_score > max_score:
                max_score = final_score
             
    return max_score/100

In [529]:
import itertools
import nltk
import numpy as np

image_dff["OCR_score"] = image_dff.apply(get_final_score, axis=1)

In [530]:
image_dff = image_dff.sort_values(by='OCR_score', ascending=False)
pd.set_option('display.max_rows', None)
len(image_dff)

469

In [531]:
RA_score = roc_auc_score(image_dff["relevant"], image_dff["OCR_score"])

print("Rekognition ROC AUC Score =  %.2f" % (RA_score))

Rekognition ROC AUC Score =  0.98


In [532]:
image_dff

,image,relevant,words,OCR_score
194,96be8e8573d42861a6adb3c5100257c451a76e5c,1,"[[among, us, you re, dumb], [99.937903881073, ...",0.999379
277,4c6bc57673884a876168e5826912ac40d40e43b7,1,"[[impostor, vwomc, n2, among, us], [99.7866570...",0.997867
49,c2d3fb2eca44bad695cd84e02c673cc27fdc175d,1,"[[impostor, vwomc, n2, among, us], [99.7304320...",0.997304
229,e0614be54bfaf19965434fef71f85b18c9a46f1c,1,"[[impostor, vwonc, n2, among, us], [99.6639847...",0.996640
84,26e418ce1e8fe6617a357e781511d226cfe45f86,1,"[[emergency, scus, meeting, apmong, us, iam, t...",0.995680
227,3fe8539d691409d2207cf5f2bd5ce453c79fe185,1,"[[among, us, impostor, among, us, impostor], [...",0.995405
72,1a660508ae3183a16538facdcbebdb17428b2a4a,1,"[[piscuss , emergency, crewme, ieeting, impos,...",0.995231
438,8b3790e98660f3b1949f77ac42542169519f23c1,1,"[[impostor], [99.52204823493958]]",0.995220
376,971e5ce3618dea173a57a79da74574d9f7a21aa4,1,"[[impostor, among, trustno, one, apmong, us, i...",0.995211
280,63094b2fcfe46bc394901b231968251c43b3e876,1,"[[impostor, vwona, n2, among, us], [99.5181322...",0.995181


In [533]:
image_dff.to_csv('/home/studio-lab-user/DS-Projects/OCR/paddle_ocr_v2.csv')

In [534]:
# Reference https://www.youtube.com/watch?v=t5xwQguk9XU

In [535]:
# Dataframe of true positives

tp = image_dff.loc[(image_dff['relevant'] == 1) & (image_dff['OCR_score'] >= 0.5)] 
print(len(tp))

119


In [536]:
# Dataframe of true negatives

tn = image_dff.loc[(image_dff['relevant'] == 0) & (image_dff['OCR_score'] < 0.5)] 
print(len(tn))

317


In [537]:
# Dataframe of false positives

fp = image_dff.loc[(image_dff['relevant'] == 0) & (image_dff['OCR_score'] >= 0.5)] 
print(len(fp))

1


In [538]:
# Dataframe of false negatives

fn = image_dff.loc[(image_dff['relevant'] == 1) & (image_dff['OCR_score'] < 0.5)] 
print(len(fn))

32


In [539]:
# Precision (What % of irdentified positives and true?)
precision = len(tp) / (len(tp) + len(fp))
print(precision)

0.9916666666666667


In [540]:
# PRecall (% of true positives identified)
recall = len(tp) / (len(tp) + len(fn))
print(recall)

0.7880794701986755


In [541]:
fn

,image,relevant,words,OCR_score
163,04437a357f2bbc9606cda30825366bb76b0559f2,1,"[[l m, your, fabher, 00000, rimong us, xvamafn...",0.499820
285,1e2688724847e6ace895252f804fc30258f92b12,1,"[[mong, whoisit, nol711], [56.05505704879761, ...",0.448440
224,9312a0394cdca0f19e08f589fb7d5d10b7b73660,1,"[[g, us, s, xvamafil, i m, your, fabher], [92....",0.410818
101,f5dd445f3fff531523efd5fa98a5b1d7700a4aed,1,"[[trick, 2, i d, skip, vote, that, kini, rgenc...",0.399637
29,d5e94eee21b701f5f3d409bb1b9bc43a4b47096c,1,"[[smang], [61.39713525772095]]",0.368383
375,587864639734fb3fbc29cfd53ff4dfdfba662acd,1,"[[smang], [61.39713525772095]]",0.368383
86,dc35ae67c9ed08081e7fe8592924ab3804840626,1,"[[wond, n2, mong, us], [81.32509589195251, 97....",0.335962
192,1f2d04f5078f7b2b6bc4e521b75e9537723c518b,1,"[[vwona, n2, among, us], [80.08106350898743, 9...",0.333739
223,9538489727bd15088eb41df30792a695eba59ad2,1,"[[wond, n2, mong, us], [78.60596776008606, 97....",0.330741
77,03613ec8794b6bf98f4ae263cafdccf6d054ee0d,1,"[[vwon , n2, among, us], [80.00211119651794, 9...",0.329893
